In [2]:
import numpy as np
from scipy import sparse
import sys
import os
import time

In [3]:
np.random.seed(0)

## Sparse .npz (compressed) takes the least disk space regardless of density
Uncompressed sparse matrices take more disk space when the matrices are very dense

In [3]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    mat = np.load(f'data/{i}.npy')
    print(i, mat.shape)
    
    density = 100 * (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density :.3} %')

    sparse_mat = sparse.csc_matrix(mat, copy=True)

    sparse.save_npz(f'data/{i}_sparse', sparse_mat)
    sparse.save_npz(f'data/{i}_sparse_un', sparse_mat, compressed=False)
    
    dense_size = os.path.getsize(f'data/{i}.npy') / 1000000
    sparse_size = os.path.getsize(f'data/{i}_sparse.npz') / 1000000
    sparse_un_size = os.path.getsize(f'data/{i}_sparse_un.npz') / 1000000
    print(f'dense: {dense_size :.4} MB\nsparse: {sparse_size :.4} MB\nsparse_uncompressed: {sparse_un_size :.4} MB\n')

CCpGeAeGaD (1391, 137)
density: 97.8 %
dense: 1.525 MB
sparse: 1.347 MB
sparse_uncompressed: 2.238 MB

DaGpBP (137, 11381)
density: 21.0 %
dense: 12.47 MB
sparse: 2.529 MB
sparse_uncompressed: 3.982 MB

MFpGdCcSE (2884, 5734)
density: 14.1 %
dense: 132.3 MB
sparse: 17.79 MB
sparse_uncompressed: 27.99 MB

GiGpBP (20945, 11381)
density: 4.65 %
dense: 1.907e+03 MB
sparse: 73.81 MB
sparse_uncompressed: 133.1 MB



## Memory-map reduces matrix-vector multiplication time for lower-density matrices

Putting a normal load above a memmap load decreases memmap load time. However, loading memmap first does not speed up normal load times. Note, this only appears to be the case for very dense matrices. For best comparisons, we should always load a memmap matrix first in the sequence.

In [4]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}.npy')
    
    density = 100 * (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density :.3} %')

#     Create a vector to multiply
    vector_size = mat.shape[0]
    vec = np.zeros((1, vector_size))
#     4 search nodes
    indices = np.random.randint(0, high=vector_size, size=4)
    vec[0, indices] = 1
    del mat
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = vec @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode=None)
    output = vec @ matrix
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
    
#     Second memory-map load time for comparison
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = vec @ matrix
    t2 = time.time()
    time_3 = t2 - t1
    del matrix
    del output
      
    print(f'Normal: {time_1 :.3} sec\nMMAP: {time_2 :.3} sec\nMMAP2: {time_3 :.3} sec\n')

CCpGeAeGaD
density: 97.8 %
Normal: 0.00106 sec
MMAP: 0.00794 sec
MMAP2: 0.000631 sec

DaGpBP
density: 21.0 %
Normal: 0.00219 sec
MMAP: 0.00109 sec
MMAP2: 0.00105 sec

MFpGdCcSE
density: 14.1 %
Normal: 0.0225 sec
MMAP: 0.0072 sec
MMAP2: 0.00725 sec

GiGpBP
density: 4.65 %
Normal: 0.31 sec
MMAP: 0.0711 sec
MMAP2: 0.0718 sec



## Sparse (uncompressed/compressed) vs Dense

Uncompressed sparse matrices load and multiply about as quickly as dense matrices. The order of matrix reads does not change for sparse reads from npz files. Compressed sparse matrices take an order of magnitude more time to load and multiply.

In [5]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}.npy')
    
    density = 100 * (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density :.3} %')

#     Create a vector to multiply
    vector_size = mat.shape[0]
    vec = np.zeros((1, vector_size))
#     4 search nodes
    indices = np.random.randint(0, high=vector_size, size=4)
    vec[0, indices] = 1
    del mat
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy')
    output = vec @ matrix
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = sparse.load_npz(f'data/{i}_sparse.npz')
    output = vec @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = sparse.load_npz(f'data/{i}_sparse_un.npz')
    output = vec @ matrix
    t2 = time.time()
    time_3 = t2 - t1
    del matrix
    del output
    
    print(f'Dense No Memmap: {time_1 :.3} sec\nCompressed Sparse: {time_2 :.3} sec\nUncompressed Sparse: {time_3 :.3} sec\n')

CCpGeAeGaD
density: 97.8 %
Dense No Memmap: 0.00634 sec
Compressed Sparse: 0.0171 sec
Uncompressed Sparse: 0.00395 sec

DaGpBP
density: 21.0 %
Dense No Memmap: 0.00218 sec
Compressed Sparse: 0.0309 sec
Uncompressed Sparse: 0.008 sec

MFpGdCcSE
density: 14.1 %
Dense No Memmap: 0.0226 sec
Compressed Sparse: 0.212 sec
Uncompressed Sparse: 0.035 sec

GiGpBP
density: 4.65 %
Dense No Memmap: 0.318 sec
Compressed Sparse: 0.983 sec
Uncompressed Sparse: 0.161 sec



## Subset matrix memmap based on nonzero rows of the search vector

In [7]:
test = np.array([1,0,1])
test_mat = np.array([[0,1,1],[1,0,0],[0,1,0]])

In [8]:
test_mat[test != 0]

array([[0, 1, 1],
       [0, 1, 0]])

In [21]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}.npy')

#     Create a vector to multiply
    vector_size = mat.shape[0]
    vec = np.zeros((1, vector_size))
#     4 search nodes
    indices = np.random.randint(0, high=vector_size, size=4)
    vec[0, indices] = 1
    del mat
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    output = vec @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = np.load(f'data/{i}.npy', mmap_mode='r')
    nonzero_rows = (vec != 0)[0]
    output = vec[0][nonzero_rows] @ matrix[nonzero_rows]
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
      
    print(f'Without subset: {time_1 :.3} sec\nWith subset: {time_2 :.3} sec\n')

CCpGeAeGaD
Without subset: 0.000396 sec
With subset: 0.00558 sec

DaGpBP
Without subset: 0.000337 sec
With subset: 0.00105 sec

MFpGdCcSE
Without subset: 0.000417 sec
With subset: 0.00646 sec

GiGpBP
Without subset: 0.034 sec
With subset: 0.0677 sec

